## CORE-MAG Mapping

Notebook loads the official MAG-CORE mapping provided by CORE and checks how many of the entries included in our corpus are mapped there

In [1]:
import pandas as pd
from pyspark import SparkContext
from glob import glob
from os.path import basename, exists, isfile, isdir, sep



matchtable = pd.read_csv('results/mag_match/2019-04-core-mag.csv.gz', compression='gzip',
                   error_bad_lines=False)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
matchtable

,coreid,magid
0,189822704,2145309562
1,34481880,2169643236
2,189822815,2036354709
3,4524506,2080979785
4,141621795,1986107543
...,...,...
8941766,78385961,2522383157
8941767,153809810,2324802629
8941768,150799886,2315671416
8941769,150672589,2333915414


In [4]:

sc = SparkContext(master = 'local[8]')
path = "/home/jovyan/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-sauer/thesis-sauer/process_2/core_sets/english_language/2021_04_07_17_27_13/batches"
if isdir(path):
    filepaths = sorted(glob(path + sep + "*"))

All Ids of the english full text CORE entries we identified are loaded.

In [ ]:
from datetime import datetime
from json import loads

print(datetime.now())
ids = []
for p in filepaths:
    print(basename(p))
    rdd = sc.textFile(f"file://{p}")
    data = rdd.map(lambda line: loads(line)['coreId'])
    ids = ids + data.collect()
    print(datetime.now())

In [18]:
s = set(matchtable.coreid)
l = [True if x in s else False for x in ids]

Number of english language entries:

In [10]:
len(ids)

6531442

Number of english entry ids included in the mapping:

In [21]:
sum(l)

654744

Load the matches we created with mag_matching.ipynb for the gnlish language entries

In [2]:
from json import load
matches = load(open('results/mag_match/matches.json'))['matches']

In [3]:
matchtable['coreid'] = matchtable['coreid'].astype('str')
matchtable['magid'] = matchtable['magid'].astype('str')

In [4]:
prematches = set([(a,b) for a,b in zip(matchtable['magid'],matchtable['coreid'])])
matches = set([(m[0],m[1]) for m in matches])

Check how many of our matches are included in the official mapping

In [5]:
inters = matches.intersection(prematches)

In [6]:
len(inters)

527429